# Ethereum BigQuery Dataset

This notebook will go over queries you can run for Ethereum, including getting top rich list, hashrate analysis, and daily gini coefficient measurement. Some queries will try to compare with other blockchain networks.

We will also be plotting our queries with [Plotly](https://plot.ly/), which we will be installing in this notebook.

In [1]:
import numpy as np
import pandas as pd
import os
from google.cloud import bigquery
!pip install plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
client = bigquery.Client()
ethereum_dataset_ref = client.dataset('crypto_ethereum', project='bigquery-public-data')

Using Kaggle's public dataset BigQuery integration.


## Top Miners By Rewards in the Last 30 Days

Here, we try to find out who are the top miners by the address of the block mined in the last 30 days. We will run the following query as a string in python through the BigQuery client.
```
WITH mined_block AS (
  SELECT miner, DATE(timestamp)
  FROM `bigquery-public-data.ethereum_classic_blockchain.blocks` 
  WHERE DATE(timestamp) > DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
  ORDER BY miner ASC)
SELECT miner, COUNT(miner) AS total_block_reward 
FROM mined_block 
GROUP BY miner 
ORDER BY total_block_reward ASC
```

Let's run it.

In [3]:
query = """
WITH mined_block AS (
  SELECT miner, DATE(timestamp)
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
  WHERE DATE(timestamp) > DATE_SUB(CURRENT_DATE(), INTERVAL 1 MONTH)
  ORDER BY miner ASC)
SELECT miner, COUNT(miner) AS total_block_reward 
FROM mined_block 
GROUP BY miner 
ORDER BY total_block_reward DESC
LIMIT 10
"""

query_job = client.query(query)
iterator = query_job.result()

In [4]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
top_miners = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
# Look at the first 10 headlines
top_miners.head(10)

,miner,total_block_reward
0,0xdf7d7e053933b5cc24372f878c90e62dadad5d42,66739
1,0x9eab4b0fc468a7f5d46228bf5a76cb52370d068d,35948
2,0x1c0fa194a9d3b44313dcd849f3c6be6ad270a0a4,22555
3,0x0073cf1b9230cf3ee8cab1971b8dbef21ea7b595,6281
4,0x58b3cabd0c5c777da2c1c4d4f7ecc8afe5674f20,4982
5,0x004730417cd2b1d19f6be2679906ded4fa8a64e2,4419
6,0xef1b68bd27237822d12e84f1449bf52ccdd4d2c1,3981
7,0xd4a56d70fb333a40f3e24b7ce8c37a75c4f489eb,2800
8,0xfe0ca4e9d8b83ff2d03ef4f35f0b5f754e81d1fd,2426
9,0xf35074bbd0a9aee46f4ea137971feec024ab704e,2351


## Plotly Library for Plotting
In this notebook, we will be using [Plotly](https://plot.ly/) for plotting our charts. You can sign up for a free account to get your API key in order to generate the charts if you choose to run this notebook on your own.

Let's start by plotting the top miners by their block reward as a pie chart.

In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [6]:
labels = top_miners['miner']
values = top_miners['total_block_reward']

trace = go.Pie(labels=labels, values=values)

iplot([trace])

## Top Miners By Block Rewards All Time

Now, let's make it more interesting and plot the total rewards of everyone who has ever mined Ethereum Classic from the genesis block.

We will limit it to just miners who's daily block rewards are greater than 100. This allows us to save on computing and plotting the traces in this Kaggle notebook.

We can use the query here:
```
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

SELECT miner, 
    DATE(timestamp) as date,
    COUNT(miner) as total_block_reward
FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
GROUP BY miner, date
HAVING COUNT(miner) > 100
ORDER BY date, COUNT(miner) ASC
```

In [7]:
query = """
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

SELECT miner, 
    DATE(timestamp) as date,
    COUNT(miner) as total_block_reward
FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
GROUP BY miner, date
HAVING COUNT(miner) > 100
ORDER BY date, COUNT(miner) ASC
"""
query_job = client.query(query)
iterator = query_job.result()

In [8]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
top_miners_by_date = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
top_miners_by_date.head(10)

,miner,date,total_block_reward
0,0x9beb4d346c6309a9ce4f27393c6322c8f03a3be6,2015-07-30,104
1,0xa1623430350c5df1b52b0b57483a5bb45d1796da,2015-07-30,130
2,0x0193d941b50d91be6567c7ee1c0fe7af498b4137,2015-07-30,147
3,0xbb12b5a9b85d4ab8cde6056e9c1b2a4a337d2261,2015-07-30,197
4,0xd7e30ae310c1d1800f5b641baa7af95b2e1fd98c,2015-07-30,205
5,0xf927a40c8b7f6e07c5af7fa2155b4864a4112b13,2015-07-30,309
6,0x1b7047b4338acf65be94c1a3e8c5c9338ad7d67c,2015-07-30,453
7,0x28921e4e2c9d84f4c0f0c0ceb991f45751a0fe93,2015-07-30,464
8,0x9dfc0377058b7b9eb277421769b56df1395705f0,2015-07-30,489
9,0xbb7b8287f3f0a933474a79eae42cbca977791171,2015-07-30,1745


In [9]:
date_series = top_miners_by_date['date'].unique()
date_series

array([datetime.date(2015, 7, 30), datetime.date(2015, 7, 31),
       datetime.date(2015, 8, 1), ..., datetime.date(2019, 2, 2),
       datetime.date(2019, 2, 3), datetime.date(2019, 2, 4)], dtype=object)

In [10]:
traces = []
miner_series = top_miners_by_date['miner'].unique()

for index, miner in enumerate(miner_series):
    miner_reward_by_date = top_miners_by_date.loc[top_miners_by_date['miner'] == miner]
    miner_reward = miner_reward_by_date['total_block_reward']
    miner_date = miner_reward_by_date['date']
    trace = dict(
        x=miner_date,
        y=miner_reward,
        mode='lines',
        stackgroup='one'
    )
    traces.append(trace)
fig = dict(data=traces)

iplot(fig)

## Daily Gini Coefficient of Ethereum Classic Mining Rewards By Miners

Now, we shall compute the [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient).

The Gini coefficient is a statistical measure of distribution used to measure income or wealth distribution among a population. From the [Investopedia article](https://www.investopedia.com/terms/g/gini-index.asp):
> A country in which every resident has the same income would have an income Gini coefficient of 0. A country in which one resident earned all the income, while everyone else earned nothing, would have an income Gini coefficient of 1.

Here, we are calculating the daily block reward distribution among addresses based on which mining address received a block daily.

The query uses what was constructed earlier and borrows from the implementation of the [Daily Balance Gini query](https://medium.com/google-cloud/calculating-gini-coefficient-in-bigquery-3bc162c82168) here. 

Also, just for fun, we shall calculate the [Simple Moving Average](https://www.investopedia.com/terms/s/sma.asp) of the Gini, using a 7 and 30 day windows. Simple Moving Average of SMA takes a set of values and their time periods, averages out a sum of the values divided by the chosen time window. 

The query value for those here will be `gini_sma_7` and `gini_sma_30`.

Please not that this only calculates the gini of those miners who earned more than 1% of the block rewards in a day, otherwise, it'll always go to 1 due to many mining just 1 block.

In [11]:
query = """
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

WITH total_reward_book AS (
  SELECT miner, 
    DATE(timestamp) as date,
    COUNT(miner) as total_block_reward
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks` 
  GROUP BY miner, date
  HAVING COUNT(miner) > 100
),
total_reward_book_by_date AS (
 SELECT date, 
        miner AS address, 
        SUM(total_block_reward / POWER(10,0)) AS value
  FROM total_reward_book
  GROUP BY miner, date
),
daily_rewards_with_gaps AS (
  SELECT
    address, 
    date,
    SUM(value) OVER (PARTITION BY ADDRESS ORDER BY date) AS block_rewards,
    LEAD(date, 1, CURRENT_DATE()) OVER (PARTITION BY ADDRESS ORDER BY date) AS next_date
  FROM total_reward_book_by_date
),
calendar AS (
  SELECT date 
  FROM UNNEST(GENERATE_DATE_ARRAY('2015-07-30', CURRENT_DATE())) AS date
),
daily_rewards AS (
  SELECT address, 
    calendar.date, 
    block_rewards
  FROM daily_rewards_with_gaps
  JOIN calendar ON daily_rewards_with_gaps.date <= calendar.date 
  AND calendar.date < daily_rewards_with_gaps.next_date
),
supply AS (
  SELECT date,
    SUM(block_rewards) AS total_rewards
  FROM daily_rewards
  GROUP BY date
),
ranked_daily_rewards AS (
  SELECT daily_rewards.date AS date,
    block_rewards,
    ROW_NUMBER() OVER (PARTITION BY daily_rewards.date ORDER BY block_rewards DESC) AS rank
  FROM daily_rewards
  JOIN supply ON daily_rewards.date = supply.date
  WHERE SAFE_DIVIDE(block_rewards, total_rewards) >= 0.01
  ORDER BY block_rewards DESC
),
daily_gini AS (
  SELECT date,
    -- (1 − 2B) https://en.wikipedia.org/wiki/Gini_coefficient
    1 - 2 * SUM((block_rewards * (rank - 1) + block_rewards / 2)) / COUNT(*) / SUM(block_rewards) AS gini
  FROM ranked_daily_rewards
  GROUP BY DATE
)
SELECT date,
  gini,
  AVG(gini) OVER (ORDER BY date ASC ROWS 7 PRECEDING) AS gini_sma_7,
  AVG(gini) OVER (ORDER BY date ASC ROWS 30 PRECEDING) AS gini_sma_30
FROM daily_gini
ORDER BY date ASC
"""

query_job = client.query(query)
iterator = query_job.result()

In [12]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
mining_reward_gini_by_date = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
mining_reward_gini_by_date.head(10)

,date,gini,gini_sma_7,gini_sma_30
0,2015-07-30,0.465213,0.465213,0.465213
1,2015-07-31,0.524871,0.495042,0.495042
2,2015-08-01,0.586563,0.525549,0.525549
3,2015-08-02,0.576192,0.538210,0.538210
4,2015-08-03,0.514817,0.533531,0.533531
5,2015-08-04,0.514918,0.530429,0.530429
6,2015-08-05,0.497703,0.525754,0.525754
7,2015-08-06,0.520803,0.525135,0.525135
8,2015-08-07,0.511116,0.530873,0.523577
9,2015-08-08,0.502773,0.528111,0.521497


In [13]:
traces = []
x = mining_reward_gini_by_date['date']
gini_list = ['gini', 'gini_sma_7', 'gini_sma_30']
for gini in gini_list:
    y = mining_reward_gini_by_date[gini]
    trace = dict(
        x=x,
        y=y,
        hoverinfo=f'{gini}',
        mode='lines'
    )
    traces.append(trace)
fig = dict(data=traces)

iplot(fig, validate=False)

## Latest Daily Balance of Ethereum Classic (Top 20 Rich List)

This next query, adapted from this [Medium post by Evgeny Medvedev](https://medium.com/google-cloud/how-to-query-balances-for-all-ethereum-addresses-in-bigquery-fb594e4034a7) will get us the latest daily balance for Ethereum Classic.

We can order it by balance, getting us a nice rich list we can plot.

In [14]:
query = """
with double_entry_book as (
    -- debits
    select to_address as address, value as value
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select miner as address, sum(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value
    from `bigquery-public-data.crypto_ethereum_classic.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum_classic.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.miner
    union all
    -- transaction fees credits
    select from_address as address, -(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value
    from `bigquery-public-data.crypto_ethereum_classic.transactions`
)
select address, 
sum(value) / 1000000000 as balance
from double_entry_book
group by address
order by balance desc
limit 20
"""

query_job = client.query(query)
iterator = query_job.result()

In [15]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
top_address_rich_list = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
top_address_rich_list.head(10)

,address,balance
0,0x6c480413ee251cb0df9606d1cb939859f85b0a93,5000000010000000
1,0x59af2dda5c57b8f17f2714bfe41232fe17a2482c,4362273652594195
2,0x149e8a848113704d0cf8c24060fdc60867680119,3469502861500000.000196
3,0x5e8f0e63e7614c47079a41ad4c37be7def06df5a,3360332283233543.895444629
4,0x8237a5caf43c8066ce4a2faca644d23b238a372c,2802449862445061.754212461
5,0xe624a238bd0890e2e1b9cfc2e846162ad1abe643,1621901498367839.545466
6,0x9f5304da62a5408416ea58a17a92611019bd5ce3,1334571732442473.158147034
7,0x13e78903cb48b593d2170c2f72bb01f41b02b033,1304356942360190.001139
8,0x9f71ddca49d311e4de8ba0c0c6220c4d0d92c92b,1251897487601960
9,0x85cc65d0456e5d1c42bfd61bca96ca317495dfe0,1246031736408310


In [16]:
labels = top_address_rich_list['address']
values = top_address_rich_list['balance']

trace = go.Pie(labels=labels, values=values)

iplot([trace])

## Daily Top Balance Gini Coefficient

Now, we will try getting daily top rich list from the genesis until now and then calculate the gini coefficient of the rich list.

In this context, the gini coefficient will be a measure of income inequality among wallet addresses based on how much ether balance is in each wallet. Of course, this assumes 1 person = 1 wallet, but a person can have multiple wallets. It will also query for top 10k addresses to be used in gini analysis. That will include exchange account balances, which we don't take into account eliminating from the dataset.

The query was written by from **[Evegeny Medvedev and Allen Day for this Google Blog Post](https://cloud.google.com/blog/products/data-analytics/introducing-six-new-cryptocurrencies-in-bigquery-public-datasets-and-how-to-analyze-them)**. I added some further analysis towards the end to measure the Simple Moving Average of the Gini for the past 7 and 30 days.

In [17]:
query = """
with 
double_entry_book as (
    -- debits
    select to_address as address, value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where to_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- credits
    select from_address as address, -value as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.traces`
    where from_address is not null
    and status = 1
    and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    union all
    -- transaction fees debits
    select miner as address, sum(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.transactions` as transactions
    join `bigquery-public-data.crypto_ethereum_classic.blocks` as blocks on blocks.number = transactions.block_number
    group by blocks.miner, block_timestamp
    union all
    -- transaction fees credits
    select from_address as address, -(cast(receipt_gas_used as numeric) * cast(gas_price as numeric)) as value, block_timestamp
    from `bigquery-public-data.crypto_ethereum_classic.transactions`
),
double_entry_book_by_date as (
    select 
        date(block_timestamp) as date, 
        address, 
        sum(value / POWER(10,0)) as value
    from double_entry_book
    group by address, date
),
daily_balances_with_gaps as (
    select 
        address, 
        date,
        sum(value) over (partition by address order by date) as balance,
        lead(date, 1, current_date()) over (partition by address order by date) as next_date
        from double_entry_book_by_date
),
calendar as (
    select date from unnest(generate_date_array('2015-07-30', current_date())) as date
),
daily_balances as (
    select address, calendar.date, balance
    from daily_balances_with_gaps
    join calendar on daily_balances_with_gaps.date <= calendar.date and calendar.date < daily_balances_with_gaps.next_date
),
 supply as (
    select
        date,
        sum(balance) as daily_supply
    from daily_balances
    group by date
),
ranked_daily_balances as (
    select 
        daily_balances.date,
        balance,
        row_number() over (partition by daily_balances.date order by balance desc) as rank
    from daily_balances
    join supply on daily_balances.date = supply.date
    where safe_divide(balance, daily_supply) >= 0.0001
    ORDER BY safe_divide(balance, daily_supply) DESC
), 
gini_daily as (
   select
    date,
    -- (1 − 2B) https://en.wikipedia.org/wiki/Gini_coefficient
    1 - 2 * sum((balance * (rank - 1) + balance / 2)) / count(*) / sum(balance) as gini
  from ranked_daily_balances
  group by date
)
select date,
    gini,
    avg(gini) over (order by date asc rows 7 preceding) as gini_sma7,
    avg(gini) over (order by date asc rows 30 preceding) as gini_sma30
from gini_daily
order by date asc
"""

query_job = client.query(query)
iterator = query_job.result()

In [18]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
daily_balance_gini = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
daily_balance_gini.head(10)

,date,gini,gini_sma7,gini_sma30
0,2015-07-30,0.689518,0.689518,0.689518
1,2015-07-31,0.689473,0.689495,0.689495
2,2015-08-01,0.689358,0.689449,0.689449
3,2015-08-02,0.689275,0.689406,0.689406
4,2015-08-03,0.689175,0.689360,0.689360
5,2015-08-04,0.689103,0.689317,0.689317
6,2015-08-05,0.689097,0.689285,0.689285
7,2015-08-06,0.689067,0.689258,0.689258
8,2015-08-07,0.697556,0.690263,0.690180
9,2015-08-08,0.702019,0.691831,0.691364


In [19]:
traces = []
x = daily_balance_gini['date']
gini_list = ['gini', 'gini_sma7', 'gini_sma30']
for gini in gini_list:
    y = daily_balance_gini[gini]
    trace = dict(
        x=x,
        y=y,
        hoverinfo=f'{gini}',
        mode='lines'
    )
    traces.append(trace)
fig = dict(data=traces)

iplot(fig, validate=False)

# Daily Hashrate

Hashrate is a measure of difficulty over block time. We can measure this by getting the delta time of each block timestamp from the previous block timestamp.

We can average it out by day. That is, the query can average out all difficulty and delta times per day and divide them by one another. We can further divide by 1 billion to get the GH/s.

We will use the following query I wrote for the Daily Hashrate.

```
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

WITH block_rows AS (
  SELECT *, ROW_NUMBER() OVER (ORDER BY timestamp) AS rn
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks`
),
delta_time AS (
  SELECT
  mp.timestamp AS block_time,
  mp.difficulty AS difficulty,
  TIMESTAMP_DIFF(mp.timestamp, mc.timestamp, SECOND) AS delta_block_time
  FROM block_rows mc
  JOIN block_rows mp
  ON mc.rn = mp.rn - 1
),
hashrate_book AS (
  SELECT TIMESTAMP_TRUNC(block_time, DAY) AS block_day,
  AVG(delta_block_time) as daily_avg_block_time,
  AVG(difficulty) as daily_avg_difficulty
  FROM delta_time
  GROUP BY TIMESTAMP_TRUNC(block_time, DAY)
)
SELECT block_day,
(daily_avg_difficulty/daily_avg_block_time)/1000000000 as hashrate
FROM hashrate_book
ORDER BY block_day ASC
```

In [20]:
query = """
#standardSQL
-- MIT License
-- Copyright (c) 2019 Yaz Khoury, yaz.khoury@gmail.com

WITH block_rows AS (
  SELECT *, ROW_NUMBER() OVER (ORDER BY timestamp) AS rn
  FROM `bigquery-public-data.crypto_ethereum_classic.blocks`
),
delta_time AS (
  SELECT
  mp.timestamp AS block_time,
  mp.difficulty AS difficulty,
  TIMESTAMP_DIFF(mp.timestamp, mc.timestamp, SECOND) AS delta_block_time
  FROM block_rows mc
  JOIN block_rows mp
  ON mc.rn = mp.rn - 1
),
hashrate_book AS (
  SELECT TIMESTAMP_TRUNC(block_time, DAY) AS block_day,
  AVG(delta_block_time) as daily_avg_block_time,
  AVG(difficulty) as daily_avg_difficulty
  FROM delta_time
  GROUP BY TIMESTAMP_TRUNC(block_time, DAY)
)
SELECT block_day,
(daily_avg_difficulty/daily_avg_block_time)/1000000000 as hashrate
FROM hashrate_book
ORDER BY block_day ASC
"""

query_job = client.query(query)
iterator = query_job.result()

In [21]:
rows = list(iterator)
# Transform the rows into a nice pandas dataframe
daily_hashrate = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
daily_hashrate.head(10)

,block_day,hashrate
0,2015-07-30 00:00:00+00:00,0.000583
1,2015-07-31 00:00:00+00:00,47.939455
2,2015-08-01 00:00:00+00:00,54.303050
3,2015-08-02 00:00:00+00:00,63.291144
4,2015-08-03 00:00:00+00:00,68.788433
5,2015-08-04 00:00:00+00:00,75.580872
6,2015-08-05 00:00:00+00:00,80.448478
7,2015-08-06 00:00:00+00:00,81.771576
8,2015-08-07 00:00:00+00:00,88.339527
9,2015-08-08 00:00:00+00:00,96.474665


In [22]:
trace = go.Scatter(
    x=daily_hashrate['block_day'],
    y=daily_hashrate['hashrate'],
    mode='lines'
)
data = [trace]
iplot(data)